In [1]:
from torchvision import transforms
import torchvision
import torch.nn as nn
import numpy as np
import torch, time, warnings, keyboard, time
warnings.filterwarnings("ignore")
from PIL import Image
from mss import mss
import matplotlib.pyplot as plt
import cv2

In [8]:
a = False
d = False

transformturn = transforms.Compose([
                        transforms.ToTensor(),                    
                        transforms.Resize([256,256]),
                        transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
                        ])

In [9]:
acc = ['w','s']
w, s = False, False
transformacc = transforms.Compose([
                    transforms.Resize([256,256]),
                    transforms.ToTensor(),
                    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
                    ])

resnetAcc = torchvision.models.resnet18(pretrained=False)
resnetAcc.fc = nn.Linear(resnetAcc.fc.in_features, len(acc))
resnetAcc.load_state_dict(torch.load('data\set1\SpeedyNet18acc.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

## ResNet18

In [ ]:
resnetTurn = resnet18(pretrained=False)
resnetTurn.fc = nn.Linear(resnetTurn.fc.in_features, len(turns))
resnetTurn.load_state_dict(torch.load('data\set2\SpeedyNet18.pth', map_location=torch.device('cpu')))
resnetTurn.eval()

## MobileNetV2

In [ ]:
modelturn = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=False, verbose=False)
modelturn.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=1280, out_features = 3)
)
modelturn.load_state_dict(torch.load('data\set2\MobileNetV2turnsCroppedimgsNoNorm.pth', map_location=torch.device('cpu')))
modelturn.eval()

## AlexNet

In [3]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 3, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

modelturn = AlexNet()
print(modelturn.load_state_dict(torch.load('data\set2\AlexNetTurnCroppedimgs.pth', map_location=torch.device('cpu'))))
modelturn.eval()

<All keys matched successfully>


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

## VGG19 batch normalization - BAD IDEA

In [ ]:
modelturn = vgg19_bn(pretrained=False)
modelturn.classifier[6] = nn.Linear(in_features=4096, out_features = 3)
print(modelturn.load_state_dict(torch.load('data\set2\VGG19_bn_crpimg.pth', map_location=torch.device('cpu'))))
modelturn.eval()

## Inception V3

In [ ]:
modelturn = torchvision.models.inception_v3(pretrained=False)
modelturn.fc = nn.Linear(in_features=2048, out_features = 3)
print(modelturn.load_state_dict(torch.load('data\set2\InceptionV3_croppedimgs.pth', map_location=torch.device('cpu'))))
modelturn.eval()

## DenseNet161

In [ ]:
modelturn = torchvision.models.densenet161(pretrained=False, num_classes=3)
print(modelturn.load_state_dict(torch.load('data\set2\DenseNet161_croppedimgs.pth', map_location=torch.device('cpu'))['weights']))
modelturn.eval()

In [4]:
def screen_record_efficient(top = 300, left = 0, width = 1920, height = 300):
  
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss()
    img = np.array(sct.grab(mon))
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    sct.close()
    return img

def flex(np_img, turnpreds,window_name = "foo"):
    global a,d
    np_img = cv2.cvtColor(np_img, cv2.COLOR_BGR2RGB)
    if turnPreds[0]>0.75: # Left
        if d:
            keyboard.release('d')
            d = False
        keyboard.press("a")
        a = True
        np_img = cv2.arrowedLine(np_img, (960, 225), (820, 225),color = (0, 0, 255), thickness= 5, tipLength=.5) 
        
    elif turnPreds[1]>0.5: # Center
        if a:
            keyboard.release('a')
            a = False
        elif d:
            keyboard.release('d')
            d = False
        np_img = cv2.arrowedLine(np_img, (960, 225), (960, 85),color = (0, 0, 255), thickness= 5, tipLength=.5) 
        
    elif turnPreds[2]>0.75: # Right
        if a:
            keyboard.release('a')
            a = False
        keyboard.press("d")
        d = True
        np_img = cv2.arrowedLine(np_img, (960, 225), (1100, 225),color = (0, 0, 255), thickness= 5, tipLength=.5)
    
    cv2.imshow(window_name, np_img) 

In [28]:
if __name__ == "__main__":
    t = True
    
    while t:
        
        try:
            foo = screen_record_efficient()
            if cv2.waitKey(1) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
            
            transformed_foo = transform(foo)
            transformed_foo = transformed_foo.unsqueeze(0)
            turnPreds = nn.functional.softmax(modelturn(transformed_foo)[0], dim=0)
            turnPreds = turnPreds.detach().numpy()
            flex(foo, turnpreds = turnPreds,window_name='speedyblackmanCAM')

        except KeyboardInterrupt:

            print("Pausing the code")
            t = input("Paused")
            if t == "q":
                t == False
            else:
                continue

In [5]:
!code .